<a href="https://colab.research.google.com/github/DaryaTereshchenko/AdClick/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from pprint import pprint

In [ ]:
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.layers import IntegerLookup, Concatenate
from tensorflow.keras.models import Sequential

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# load data
from urllib.request import urlretrieve
import os

def download(url, file):
    if not os.path.isfile(file):
        print("Download file... " + file + " ...")
        urlretrieve(url,file)
        print("File downloaded")

d100k_url = "https://home.ipipan.waw.pl/sj/Warsztaty/AdClick/D10M.tsv.gz"
download(d100k_url,'D10M.tsv.gz')
print("All the files are downloaded")

All the files are downloaded


In [ ]:
ds = tf.data.experimental.make_csv_dataset("D10M.tsv.gz",
                                           field_delim="\t",
                                           compression_type="GZIP",
                                           num_epochs=1,
                                           batch_size=128,  # increase for real work
                                           label_name=None, shuffle=False)


In [ ]:
pprint(ds.element_spec)

OrderedDict([('Click', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
             ('DisplayURL',
              TensorSpec(shape=(None,), dtype=tf.float32, name=None)),
             ('AdId', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
             ('AdvertiserId',
              TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
             ('Depth', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
             ('Position', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
             ('UserID', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
             ('Gender', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
             ('Age', TensorSpec(shape=(None,), dtype=tf.int32, name=None)),
             ('AdKeyword_tokens',
              TensorSpec(shape=(None,), dtype=tf.string, name=None)),
             ('AdTitle_tokens',
              TensorSpec(shape=(None,), dtype=tf.string, name=None)),
             ('AdDescription_tokens',
          

In [ ]:
print(type(ds))

<class 'tensorflow.python.data.ops.dataset_ops.PrefetchDataset'>


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

NameError: ignored

In [ ]:
pprint(next(iter(ds)))

In [ ]:
inputs = {
    "Position": Input(shape=(), dtype=tf.int32),
    "Depth": Input(shape=(), dtype=tf.int32),
    "Gender": Input(shape=(), dtype=tf.int32)
}

In [ ]:
# from tensorflow.keras.layers import CategoryEncoding
# from tensorflow.keras.layers import StringLookup
# from tensorflow.keras.layers import Hashing

In [ ]:
ci_p =  IntegerLookup(vocabulary=[1,2,3], output_mode="one_hot", num_oov_indices=0)(inputs["Position"])
ci_d = IntegerLookup(vocabulary=[1,2,3], output_mode="one_hot", num_oov_indices=0)(inputs["Depth"])
ci_g = IntegerLookup(vocabulary=[0,1,2], output_mode="one_hot", num_oov_indices=0)(inputs["Gender"])

In [ ]:
encoded = Concatenate()([ci_p, ci_d, ci_g])

In [ ]:
encoding_model = Model(inputs, encoded)


<tf.Tensor: shape=(128, 9), dtype=float32, numpy=
array([[0., 0., 1., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [0., 1., 0., ..., 0., 1., 0.],
       ...,
       [0., 1., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.]], dtype=float32)>

In [ ]:
encoding_model(next(iter(ds))[0]) # take only fearues, not class

In [ ]:
# simple logistic regression
y = Dense(1, activation="sigmoid")(encoded)
m = Model(inputs, y)

In [ ]:
m.compile(optimizer="Adam", loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
          metrics=["AUC"])

In [ ]:
m.fit(ds.unbatch().batch(128), epochs=20)

Epoch 1/20
782/782 [==============================] - 6s 6ms/step - loss: 0.3246 - auc: 0.4739
Epoch 2/20
782/782 [==============================] - 5s 6ms/step - loss: 0.2094 - auc: 0.4637
Epoch 3/20
782/782 [==============================] - 5s 6ms/step - loss: 0.1990 - auc: 0.4757
Epoch 4/20
782/782 [==============================] - 5s 6ms/step - loss: 0.1960 - auc: 0.5088
Epoch 5/20
782/782 [==============================] - 5s 6ms/step - loss: 0.1945 - auc: 0.5611
Epoch 6/20
782/782 [==============================] - 5s 6ms/step - loss: 0.1937 - auc: 0.5788
Epoch 7/20
782/782 [==============================] - 5s 6ms/step - loss: 0.1933 - auc: 0.5896
Epoch 8/20
782/782 [==============================] - 5s 7ms/step - loss: 0.1931 - auc: 0.5927
Epoch 9/20
782/782 [==============================] - 6s 7ms/step - loss: 0.1930 - auc: 0.5944
Epoch 10/20
782/782 [==============================] - 5s 7ms/step - loss: 0.1930 - auc: 0.5952
Epoch 11/20
782/782 [============================

In [ ]:
m.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 input_5 (InputLayer)           [(None,)]            0           []                               
                                                                                                  
 integer_lookup_19 (IntegerLook  (None, 3)           0           ['input_3[0][0]']                
 up)                                                                                        

In [ ]:
def compile_and_run(model, epochs=10):
    model.compile(loss=tf.keras.losses.binary_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
    print(model.layers)
    model.summary()
    batch_size = 128
    model.fit(data_frame, labels,
              batch_size=batch_size,
              epochs=epochs,
              verbose=1,
              validation_data=(x_test_p, y_test))
    # score = model.evaluate(x_test_p, y_test, verbose=0)
    # print('Test loss:', score[0])
    # print('Test accuracy:', score[1])
    return tf.keras.utils.plot_model(model)

In [ ]:
model2 = Sequential()
